In [1]:
import os
import sys
sys.path.append('../Automatic-Circuit-Discovery/')
sys.path.append('..')
import re
import IPython
ipython = get_ipython()
if ipython is not None:
    ipython.magic('load_ext autoreload')
    ipython.magic('autoreload 2')
import acdc
from acdc.TLACDCExperiment import TLACDCExperiment
from acdc.acdc_utils import TorchIndex, EdgeType
import numpy as np
import torch as t
from torch import Tensor
import einops
import itertools

from transformer_lens import HookedTransformer, ActivationCache

import tqdm.notebook as tqdm
import plotly
from rich import print as rprint
from rich.table import Table

from jaxtyping import Float, Bool
from typing import Callable, Tuple, Union, Dict, Optional

device = t.device('cuda') if t.cuda.is_available() else t.device('cpu')
print(f'Device: {device}')

/tmp/ipykernel_26936/341448825.py:9: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic('load_ext autoreload')
/tmp/ipykernel_26936/341448825.py:10: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic('autoreload 2')


Device: cuda


# Model Setup

In [2]:
model = HookedTransformer.from_pretrained(
    'gpt2-small',
    center_writing_weights=False,
    center_unembed=False,
    fold_ln=False,
    device=device,
)
model.set_use_hook_mlp_in(True)
model.set_use_split_qkv_input(True)
model.set_use_attn_result(True)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


# Dataset Setup

In [3]:
from ioi_dataset import IOIDataset, format_prompt, make_table
N = 25
clean_dataset = IOIDataset(
    prompt_type='mixed',
    N=N,
    tokenizer=model.tokenizer,
    prepend_bos=False,
    seed=1,
    device=device
)
corr_dataset = clean_dataset.gen_flipped_prompts('ABC->XYZ, BAB->XYZ')

make_table(
  colnames = ["IOI prompt", "IOI subj", "IOI indirect obj", "ABC prompt"],
  cols = [
    map(format_prompt, clean_dataset.sentences),
    model.to_string(clean_dataset.s_tokenIDs).split(),
    model.to_string(clean_dataset.io_tokenIDs).split(),
    map(format_prompt, clean_dataset.sentences),
  ],
  title = "Sentences from IOI vs ABC distribution",
)

                                      Sentences from IOI vs ABC distribution                                       
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ IOI prompt                              ┃ IOI subj ┃ IOI indirect obj ┃ ABC prompt                              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ When Victoria and Jane got a snack at   │ Jane     │ Victoria         │ When Victoria and Jane got a snack at   │
│ the store, Jane decided to give it to   │          │                  │ the store, Jane decided to give it to   │
│ Victoria                                │          │                  │ Victoria                                │
│                                         │          │                  │                                         │
│ When Sullivan and Rose got a necklace   │ Sullivan │ Rose             │ When Sullivan and Rose got a necklace   │
│ at the garden, Sullivan decided to give │          │                  │ at the garden, Sullivan decided to give │
│ it to Rose                              │          │                  │ it to Rose                              │
│                                         │          │                  │                                         │
│ When Alan and Alex got a drink at the   │ Alex     │ Alan             │ When Alan and Alex got a drink at the   │
│ store, Alex decided to give it to Alan  │          │                  │ store, Alex decided to give it to Alan  │
│                                         │          │                  │                                         │
│ Then, Jessica and Crystal had a long    │ Jessica  │ Crystal          │ Then, Jessica and Crystal had a long    │
│ argument, and afterwards Jessica said   │          │                  │ argument, and afterwards Jessica said   │
│ to Crystal                              │          │                  │ to Crystal                              │
│                                         │          │                  │                                         │
│ Then, Jonathan and Kevin were working   │ Kevin    │ Jonathan         │ Then, Jonathan and Kevin were working   │
│ at the school. Kevin decided to give a  │          │                  │ at the school. Kevin decided to give a  │
│ necklace to Jonathan                    │          │                  │ necklace to Jonathan                    │
│                                         │          │                  │                                         │
└─────────────────────────────────────────┴──────────┴──────────────────┴─────────────────────────────────────────┘

# Metric Setup

In [7]:
def ave_logit_diff(
    logits: Float[Tensor, 'batch seq d_vocab'],
    ioi_dataset: IOIDataset,
    per_prompt: bool = False
):
    '''
        Return average logit difference between correct and incorrect answers
    '''
    # Get logits for indirect objects
    io_logits = logits[range(logits.size(0)), ioi_dataset.word_idx['end'], ioi_dataset.io_tokenIDs]
    s_logits = logits[range(logits.size(0)), ioi_dataset.word_idx['end'], ioi_dataset.s_tokenIDs]
    # Get logits for subject
    logit_diff = io_logits - s_logits
    return logit_diff if per_prompt else logit_diff.mean()

with t.no_grad():
    clean_logits = model(clean_dataset.toks)
    corrupt_logits = model(corr_dataset.toks)
    clean_logit_diff = ave_logit_diff(clean_logits, clean_dataset).item()
    corrupt_logit_diff = ave_logit_diff(corrupt_logits, corr_dataset).item()

def ioi_metric(
    logits: Float[Tensor, "batch seq_len d_vocab"],
    corrupted_logit_diff: float = corrupt_logit_diff,
    clean_logit_diff: float = clean_logit_diff,
    ioi_dataset: IOIDataset = clean_dataset
 ):
    patched_logit_diff = ave_logit_diff(logits, ioi_dataset)
    return (patched_logit_diff - corrupted_logit_diff) / (clean_logit_diff - corrupted_logit_diff)

def negative_ioi_metric(logits: Float[Tensor, "batch seq_len d_vocab"]):
    return -ioi_metric(logits)
    
# Get clean and corrupt logit differences
with t.no_grad():
    clean_metric = ioi_metric(clean_logits, corrupt_logit_diff, clean_logit_diff, clean_dataset)
    corrupt_metric = ioi_metric(corrupt_logits, corrupt_logit_diff, clean_logit_diff, corr_dataset)

print(f'Clean direction: {clean_logit_diff}, Corrupt direction: {corrupt_logit_diff}')
print(f'Clean metric: {clean_metric}, Corrupt metric: {corrupt_metric}')

Clean direction: 2.8051609992980957, Corrupt direction: 1.7753835916519165
Clean metric: 1.0, Corrupt metric: 0.0


# Run Experiment

In [10]:
from ACDCPPExperiment import ACDCPPExperiment
THRESHOLDS = np.arange(0.005, 0.155, 0.005)
RUN_NAME = 'noabs_value'
acdcpp_exp = ACDCPPExperiment(model,
    clean_dataset.toks,
    corr_dataset.toks,
    negative_ioi_metric, 
    negative_ioi_metric, # Arthur changed both to negative_ioi_metric as otherwise it was finding the negative things...
    THRESHOLDS,
    run_name=RUN_NAME,
    verbose=True,
    attr_absolute_val=True,
    save_graphs_after=0.07,
    pruning_mode="edge",
)
pruned_heads, num_passes, pruned_attrs = acdcpp_exp.run()

Set up model hooks


  0%|          | 0/30 [00:00<?, ?it/s]

dict_keys(['blocks.11.hook_resid_post', 'blocks.11.hook_mlp_out', 'blocks.11.hook_mlp_in', 'blocks.11.attn.hook_result', 'blocks.11.attn.hook_q', 'blocks.11.hook_q_input', 'blocks.11.attn.hook_k', 'blocks.11.hook_k_input', 'blocks.11.attn.hook_v', 'blocks.11.hook_v_input', 'blocks.10.hook_mlp_out', 'blocks.10.hook_mlp_in', 'blocks.10.attn.hook_result', 'blocks.10.attn.hook_q', 'blocks.10.hook_q_input', 'blocks.10.attn.hook_k', 'blocks.10.hook_k_input', 'blocks.10.attn.hook_v', 'blocks.10.hook_v_input', 'blocks.9.hook_mlp_out', 'blocks.9.hook_mlp_in', 'blocks.9.attn.hook_result', 'blocks.9.attn.hook_q', 'blocks.9.hook_q_input', 'blocks.9.attn.hook_k', 'blocks.9.hook_k_input', 'blocks.9.attn.hook_v', 'blocks.9.hook_v_input', 'blocks.8.hook_mlp_out', 'blocks.8.hook_mlp_in', 'blocks.8.attn.hook_result', 'blocks.8.attn.hook_q', 'blocks.8.hook_q_input', 'blocks.8.attn.hook_k', 'blocks.8.hook_k_input', 'blocks.8.attn.hook_v', 'blocks.8.hook_v_input', 'blocks.7.hook_mlp_out', 'blocks.7.hook_ml

NOT PRUNING upstream_component=ModelComponent(hook_point_name='blocks.11.attn.hook_result', index=[:, :, 11]) downstream_component=ModelComponent(hook_point_name='blocks.11.hook_resid_post', index=[:]) with attribution 0.01751333847641945
NOT PRUNING upstream_component=ModelComponent(hook_point_name='blocks.11.attn.hook_result', index=[:, :, 11]) downstream_component=ModelComponent(hook_point_name='blocks.11.hook_mlp_in', index=[:]) with attribution 0.007915440946817398
NOT PRUNING upstream_component=ModelComponent(hook_point_name='blocks.11.attn.hook_result', index=[:, :, 10]) downstream_component=ModelComponent(hook_point_name='blocks.11.hook_resid_post', index=[:]) with attribution 0.6607147455215454


NOT PRUNING upstream_component=ModelComponent(hook_point_name='blocks.11.attn.hook_result', index=[:, :, 10]) downstream_component=ModelComponent(hook_point_name='blocks.11.hook_mlp_in', index=[:]) with attribution 0.0076356856152415276
NOT PRUNING upstream_component=ModelComponent(hook_point_name='blocks.11.attn.hook_result', index=[:, :, 9]) downstream_component=ModelComponent(hook_point_name='blocks.11.hook_resid_post', index=[:]) with attribution 0.04760678857564926
NOT PRUNING upstream_component=ModelComponent(hook_point_name='blocks.11.attn.hook_result', index=[:, :, 9]) downstream_component=ModelComponent(hook_point_name='blocks.11.hook_mlp_in', index=[:]) with attribution 0.008992362767457962
NOT PRUNING upstream_component=ModelComponent(hook_point_name='blocks.11.attn.hook_result', index=[:, :, 8]) downstream_component=ModelComponent(hook_point_name='blocks.11.hook_mlp_in', index=[:]) with attribution 0.015257244929671288
NOT PRUNING upstream_component=ModelComponent(hook_poin

...

NOT PRUNING upstream_component=ModelComponent(hook_point_name='blocks.2.attn.hook_result', index=[:, :, 11]) downstream_component=ModelComponent(hook_point_name='blocks.6.hook_q_input', index=[:, :, 9]) with attribution 0.00569714093580842


NOT PRUNING upstream_component=ModelComponent(hook_point_name='blocks.2.attn.hook_result', index=[:, :, 11]) downstream_component=ModelComponent(hook_point_name='blocks.5.hook_mlp_in', index=[:]) with attribution 0.009011846967041492


NOT PRUNING upstream_component=ModelComponent(hook_point_name='blocks.2.attn.hook_result', index=[:, :, 11]) downstream_component=ModelComponent(hook_point_name='blocks.2.hook_mlp_in', index=[:]) with attribution 0.009746287018060684
NOT PRUNING upstream_component=ModelComponent(hook_point_name='blocks.2.attn.hook_result', index=[:, :, 10]) downstream_component=ModelComponent(hook_point_name='blocks.11.hook_k_input', index=[:, :, 10]) with attribution 0.007407147437334061


NOT PRUNING upstream_component=ModelComponent(hook_point_name='blocks.2.attn.hook_result', index=[:, :, 10]) downstream_component=ModelComponent(hook_point_name='blocks.9.hook_k_input', index=[:, :, 7]) with attribution 0.007820762693881989


NOT PRUNING upstream_component=ModelComponent(hook_point_name='blocks.2.attn.hook_result', index=[:, :, 10]) downstream_component=ModelComponent(hook_point_name='blocks.9.hook_q_input', index=[:, :, 7]) with attribution 0.005306468345224857


# Save Data

In [ ]:
import json

for thresh in pruned_heads.keys():
   pruned_heads[thresh][0] = list(pruned_heads[thresh][0])
   pruned_heads[thresh][1] = list(pruned_heads[thresh][1])

cleaned_attrs = {}
for thresh in pruned_attrs.keys():
    cleaned_attrs[thresh] = {}
    for (layer, head), attr in pruned_attrs[thresh].items():
        cleaned_attrs[thresh][f'L{layer}H{head}'] = attr
        
with open(f'{RUN_NAME}_pruned_heads.json', 'w') as f:
    json.dump(pruned_heads, f)
with open(f'{RUN_NAME}_num_passes.json', 'w') as f:
    json.dump(num_passes, f)
with open(f'{RUN_NAME}_pruned_attrs.json', 'w') as f:
    json.dump(cleaned_attrs, f)